In [1]:
!pip install transformers
!pip install datasets
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 1.7 MB

In [2]:
from google.colab import userdata
import os
from huggingface_hub import login
import pandas as pd
import datasets
from datasets import load_dataset,Dataset


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
from google.colab import files
import pandas as pd

ruta = "/content/drive/MyDrive/Colab Notebooks/DCM/Trabajo Final/Data/"
libros_lista = []
for i in os.listdir(ruta):
  libro = pd.read_csv(ruta + str(i),sep="\t",header=None,names=['texto'])
  # libro['libro'] = i
  libros_lista.append(libro)

In [19]:
libros = pd.concat(libros_lista, axis=0)

In [20]:
libros.shape

(36614, 1)

In [43]:
libros_train = libros.iloc[:25629,:]
libros_val = libros.iloc[25629:,:]

In [21]:
libros = libros.set_index(['libro'], drop=True, append=False, inplace=False, verify_integrity=False)

KeyError: "None of ['libro'] are in the columns"

In [22]:
libros

,texto
0,"M r. and Mrs. Dursley, of number four, Privet ..."
1,Mr. Dursley was the director of a firm called ...
2,"The Dursleys had everything they wanted, but t..."
3,"When Mr. and Mrs. Dursley woke up on the dull,..."
4,"None of them noticed a large, tawny owl flutte..."
...,...
9287,"`Take care, Harry,' said Lupin quietly. `Keep ..."
9288,"`Harry, we'll have you away from there as soon..."
9289,"`We'll see you soon, mate,' said Ron anxiously..."
9290,"`Really soon, Harry' said Hermione earnestly. ..."


In [44]:
dataset = Dataset.from_pandas(libros)

In [24]:
print(dataset)

Dataset({
    features: ['texto', '__index_level_0__'],
    num_rows: 36614
})


In [25]:
from transformers import AutoConfig

# Download configuration from S3 and cache.
# config = AutoConfig.from_pretrained('bert-base-uncased')

# Download configuration from S3 (user-uploaded) and cache.
# config = AutoConfig.from_pretrained('dbmdz/bert-base-german-cased')

# If configuration file is in a directory (e.g., was saved using `save_pretrained('./test/saved_model/')`).
# config = AutoConfig.from_pretrained('./test/bert_saved_model/')

# Load a specific configuration file.
# config = AutoConfig.from_pretrained('./test/bert_saved_model/my_configuration.json')

# Change some config attributes when loading a pretrained config.
config = AutoConfig.from_pretrained('bert-base-uncased', output_attentions=True, foo=False)
# config.output_attentions
# config, unused_kwargs = AutoConfig.from_pretrained('bert-base-uncased', output_attentions=True, foo=False, return_unused_kwargs=True)
# config.output_attentions
# config.unused_kwargs

In [30]:
# Integrar el modelo PEFT
# model = PeftModel(base_model, peft_config)
# Cargar los libros como dataset de texto
#dataset = load_dataset("text", data_files=data_files)

# Función para tokenizar el dataset
def tokenize_function(examples):
  return tokenizer(examples['texto'], truncation=True, padding="max_length", return_tensors='pt')

# # Tokenizar el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Agregar las etiquetas al dataset tokenizado
def add_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples

# Aplicar la función de etiquetas
tokenized_datasets = tokenized_datasets.map(add_labels, batched=True)

Map:   0%|          | 0/36614 [00:00<?, ? examples/s]

Map:   0%|          | 0/36614 [00:00<?, ? examples/s]

In [33]:
from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification
from transformers import BertModel

# Cargar el modelo Meta-Llama 3.1 8B en modo 8 bits
# model = model = AutoModel.from_pretrained('bert-base-uncased',num_labels=num_labels)
# from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=512)

from peft import PeftModel, PeftConfig

# Configuración del modelo PEFT
peft_config = PeftConfig(
    base_model_name_or_path='bert-base-uncased'
    # adapter_name='my_adapter'
)

# Cargar el modelo base
base_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')





Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Asegúrate de que tu dataset sea una instancia de Dataset
assert isinstance(tokenized_datasets, Dataset)

In [ ]:
tokenized_datasets[:25629]

In [48]:
tokenized_datasets_train = tokenized_datasets[:25629]
# tokenized_datasets_val = tokenized_datasets.iloc[25629:,:]

In [56]:
tokenized_datasets[0]

{'texto': 'M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.',
 '__index_level_0__': 0,
 'input_ids': [101,
  1049,
  1054,
  1012,
  1998,
  3680,
  1012,
  4241,
  2869,
  3051,
  1010,
  1997,
  2193,
  2176,
  1010,
  26927,
  19510,
  3298,
  1010,
  2020,
  7098,
  2000,
  2360,
  2008,
  2027,
  2020,
  6669,
  3671,
  1010,
  4067,
  2017,
  2200,
  2172,
  1012,
  2027,
  2020,
  1996,
  2197,
  2111,
  2017,
  1521,
  1040,
  5987,
  2000,
  2022,
  2920,
  1999,
  2505,
  4326,
  2030,
  8075,
  1010,
  2138,
  2027,
  2074,
  2134,
  1521,
  1056,
  2907,
  2007,
  2107,
  14652,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [55]:
tokenized_datasets_train[1]

KeyError: 1

In [38]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=28,
    per_device_eval_batch_size=28,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    fp16=True,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Asegúrate de que esto sea un Dataset
    eval_dataset=tokenized_datasets,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 13.06 MiB is free. Process 9406 has 14.73 GiB memory in use. Of the allocated memory 14.38 GiB is allocated by PyTorch, and 231.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Esta parte del codigo es para agregarle un guardado automatico a los step

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     gradient_accumulation_steps=8,
#     num_train_epochs=3,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     fp16=True,
#     save_total_limit=2,
#     save_steps=500,  # Guardar cada 500 pasos
#     save_strategy="steps"  # Usar la estrategia de guardar por pasos
# )

In [ ]:
trainer.save_model("./fine-tuned-llama-harry-potter")


Esta parte del codigo es para reanudarlo

In [ ]:
# # Cargar el modelo desde el último checkpoint
# model = AutoModelForCausalLM.from_pretrained("./checkpoint")

# # Continuar el entrenamiento
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["train"],
# )

# trainer.train(resume_from_checkpoint=True)

Probando modelo original

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargar el modelo original (antes del fine-tuning)
model_original = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
tokenizer_original = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

# Asegurar que el tokenizador tiene el pad_token asignado
tokenizer_original.pad_token = tokenizer_original.eos_token

# Texto de prueba
input_text = "Harry Potter went to"

# Tokenizar el texto de entrada
inputs = tokenizer_original(input_text, return_tensors="pt")

# Generar texto con el modelo original
outputs_original = model_original.generate(**inputs, max_length=50)
generated_text_original = tokenizer_original.decode(outputs_original[0], skip_special_tokens=True)

print("Texto generado por el modelo original:")
print(generated_text_original)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

descargar una carpeta entera

In [ ]:
# Comprimir la carpeta 'results' en un archivo zip
!zip -r results.zip /content/results


  adding: content/results/ (stored 0%)
  adding: content/results/runs/ (stored 0%)
  adding: content/results/runs/Sep13_20-42-52_159f47a53ae5/ (stored 0%)
  adding: content/results/runs/Sep13_20-42-52_159f47a53ae5/events.out.tfevents.1726260172.159f47a53ae5.614.5 (deflated 62%)
  adding: content/results/runs/Sep13_20-32-38_159f47a53ae5/ (stored 0%)
  adding: content/results/runs/Sep13_20-32-38_159f47a53ae5/events.out.tfevents.1726259560.159f47a53ae5.614.0 (deflated 62%)
  adding: content/results/runs/Sep13_20-36-35_159f47a53ae5/ (stored 0%)
  adding: content/results/runs/Sep13_20-36-35_159f47a53ae5/events.out.tfevents.1726259796.159f47a53ae5.614.4 (deflated 62%)
  adding: content/results/runs/Sep13_20-36-17_159f47a53ae5/ (stored 0%)
  adding: content/results/runs/Sep13_20-36-17_159f47a53ae5/events.out.tfevents.1726259778.159f47a53ae5.614.3 (deflated 62%)
  adding: content/results/runs/Sep13_20-35-57_159f47a53ae5/ (stored 0%)
  adding: content/results/runs/Sep13_20-35-57_159f47a53ae5/ev

In [ ]:
from google.colab import files
files.download('results.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Comprimir la carpeta 'results' en un archivo zip
!zip -r fine-tuned-llama-harry-potter.zip /content/fine-tuned-llama-harry-potter

from google.colab import files
files.download('fine-tuned-llama-harry-potter.zip')

  adding: content/fine-tuned-llama-harry-potter/ (stored 0%)
  adding: content/fine-tuned-llama-harry-potter/training_args.bin (deflated 51%)
  adding: content/fine-tuned-llama-harry-potter/README.md (deflated 66%)
  adding: content/fine-tuned-llama-harry-potter/adapter_model.safetensors (deflated 8%)
  adding: content/fine-tuned-llama-harry-potter/adapter_config.json (deflated 51%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls /root/.cache/huggingface/transformers


ls: cannot access '/root/.cache/huggingface/transformers': No such file or directory


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Cargar el modelo fine-tuned usando LoRA
model_finetuned = AutoModelForCausalLM.from_pretrained("./fine-tuned-llama-harry-potter")  # Ruta donde guardaste el modelo entrenado

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-llama-harry-potter")  # Ruta del tokenizador guardado junto al modelo
tokenizer.pad_token = tokenizer.eos_token

# Texto de entrada para probar
input_text = "Harry Potter went to"

# Tokenizar el texto de entrada
inputs = tokenizer(input_text, return_tensors="pt")

# Generar texto con el modelo fine-tuned
outputs = model_finetuned.generate(**inputs, max_length=5) # Era 50 y se queda en 25%

# Decodificar los tokens generados a texto legible
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Texto generado por el modelo entrenado (fine-tuned):")
print(generated_text)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!pwd